<a href="https://colab.research.google.com/github/JulieaMammen/Education-Inequality/blob/main/Julie_Mammen_DATA_3320_Education_Inequality_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

The purpose of this notebook is to produce a joined data frame of the edGap data and the school information data to see if there is a relationship between school performance in relation to socioeconomic factors.

## Import libraries

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split

# Model preprocessing
from sklearn.preprocessing import StandardScaler

# Modeling
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Model metrics and analysis
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

## The data

This project utilizes two data sets. The primary data set is the EdGap data set from [EdGap.org](https://www.edgap.org/#5/37.875/-96.987). This data set from 2016 includes information about average ACT or SAT scores for schools and several socioeconomic characteristics of the school district. The secondary data set is basic information about each school from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp).





### EdGap data

All socioeconomic data (household income, unemployment, adult educational attainment, and family structure) are from the Census Bureau's American Community Survey. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) report that ACT and SAT score data is from each state's department of education or some other public data release. The nature of the other public data release is not known.

The quality of the census data and the department of education data can be assumed to be reasonably high. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) do not indicate that they processed the data in any way. The data were assembled by the [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) team, so there is always the possibility for human error. Given the public nature of the data, we would be able to consult the original data sources to check the quality of the data if we had any questions.

### School information data

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about schools and can be assumed to be of reasonably high quality. As for the EdGap.org data, the school information data is public, so we would be able to consult the original data sources to check the quality of the data if we had any questions.


## Load the data

Load the EdGap
 data set

In [2]:
edgap = pd.read_excel('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Load the school information data

In [3]:
#!wget https://www.dropbox.com/s/dz2dqbvwctsde6f/ccd_sch_029_1617_w_1a_11212017.csv?dl=0 
!wget https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0

--2023-04-25 03:29:42--  https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv [following]
--2023-04-25 03:29:42--  https://www.dropbox.com/s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3ace4f486b1f057d3e4a5d3bbe.dl.dropboxusercontent.com/cd/0/inline/B63sBAQaZJ17eGCZpWCk_23oU2uKEq8J-Uisu8yhuR-HILseVLiYEYtqwOaBXWobhZIT-Yc95GaWiwoLJxfbA-XjiFRBLXlMgvGsW_72Lh7oirpxGvWGQNLsIIRMDnYM5j_mpk-edmwK235so9IT8y9izkXVdZch7t9rhpQI3__IUQ/file# [following]
--2023-04-25 03:29:43--  https://uc3ace4f486b1f057d3e4a5d3bbe.dl.dropboxusercontent.com/cd/0/inline/B63sBAQaZ

In [4]:
school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding= 'unicode_escape')

<ipython-input-4-54e0747c4ccf>:1: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding= 'unicode_escape')


## Explore the contents of the data sets

We start by looking at the head of each data frame. This will let us see the names of the columns and a few example values for each column.

In [5]:
edgap.head()

,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [6]:
school_info.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported


We can use the `info` method to check the data types, size of the data frame, and numbers of missing values.

In [7]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   NCESSCH School ID                                7986 non-null   int64  
 1   CT Unemployment Rate                             7972 non-null   float64
 2   CT Pct Adults with College Degree                7973 non-null   float64
 3   CT Pct Childre In Married Couple Family          7961 non-null   float64
 4   CT Median Household Income                       7966 non-null   float64
 5   School ACT average (or equivalent if SAT score)  7986 non-null   float64
 6   School Pct Free and Reduced Lunch                7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


There are some missing observations, but not many. However, this does not address school data that is missing from the data set.

In [8]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102183 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102183 non-null  object 
 1   FIPST                102183 non-null  int64  
 2   STATENAME            102183 non-null  object 
 3   ST                   102183 non-null  object 
 4   SCH_NAME             102183 non-null  object 
 5   LEA_NAME             102183 non-null  object 
 6   STATE_AGENCY_NO      102183 non-null  object 
 7   UNION                2533 non-null    float64
 8   ST_LEAID             102183 non-null  object 
 9   LEAID                102183 non-null  object 
 10  ST_SCHID             102183 non-null  object 
 11  NCESSCH              102181 non-null  float64
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

This data set is much larger than the EdGap data set. Indicating that the EdGap data set does not include all schools.

There are many null values in the school information data set, but we aren't concerned about all of the columns. 

## Convert data types, if necessary

We will want to join the DataFrames using the identity of the school as the key. The identity is given by the NCESSCH school identity, which has a different name in the two DataFrames, and the value is an `int64` in the EdGap data set and a `float64` in the school information data set.

We will cast the `NCESSCH` column in the `school_info` DataFrame as an `int64`. We first need to drop rows where `NCESSCH` is a `NaN` value. 

In [9]:
school_info = school_info[school_info['NCESSCH'].isna() == False]

In [10]:
school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')

<ipython-input-10-006ea9997674>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')


In [11]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102181 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102181 non-null  object 
 1   FIPST                102181 non-null  int64  
 2   STATENAME            102181 non-null  object 
 3   ST                   102181 non-null  object 
 4   SCH_NAME             102181 non-null  object 
 5   LEA_NAME             102181 non-null  object 
 6   STATE_AGENCY_NO      102181 non-null  object 
 7   UNION                2531 non-null    float64
 8   ST_LEAID             102181 non-null  object 
 9   LEAID                102181 non-null  object 
 10  ST_SCHID             102181 non-null  object 
 11  NCESSCH              102181 non-null  int64  
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

## Select relevant subsets of the data

The school information data set contains a lot of information. However, we only need the year, school identity, location, and school type information.

So we will keep the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL`

In [12]:
school_info = school_info[['SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']]

In [13]:
school_info.head()

,SCHOOL_YEAR,NCESSCH,MSTATE,MZIP,SCH_TYPE_TEXT,LEVEL
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


## Rename columns

In [14]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id", 
              "CT Pct Adults with College Degree":"percent_college",        
              "CT Unemployment Rate":"rate_unemployment", 
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})

In [15]:
school_info = school_info.rename(columns={'SCHOOL_YEAR':'year', 
                                          'NCESSCH':'id', 
                                          'MSTATE':'state',
                                          'MZIP':'zip_code',
                                          'SCH_TYPE_TEXT':'school_type',
                                          'LEVEL':'school_level'})

In [16]:
edgap.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [17]:
school_info.head()

,year,id,state,zip_code,school_type,school_level
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


## Join data frames 

In [18]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


In [19]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102181 entries, 0 to 102182
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   year          102181 non-null  object
 1   id            102181 non-null  int64 
 2   state         102181 non-null  object
 3   zip_code      102181 non-null  object
 4   school_type   102179 non-null  object
 5   school_level  102179 non-null  object
dtypes: int64(1), object(5)
memory usage: 5.5+ MB


In [20]:
df = edgap.merge(school_info, how = 'left', on='id')

In [21]:
df.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7986 entries, 0 to 7985
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7986 non-null   int64  
 1   rate_unemployment  7972 non-null   float64
 2   percent_college    7973 non-null   float64
 3   percent_married    7961 non-null   float64
 4   median_income      7966 non-null   float64
 5   average_act        7986 non-null   float64
 6   percent_lunch      7986 non-null   float64
 7   year               7898 non-null   object 
 8   state              7898 non-null   object 
 9   zip_code           7898 non-null   object 
 10  school_type        7898 non-null   object 
 11  school_level       7898 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 811.1+ KB


## Quality Control: Check for out-of-range values

There are natural bounds for the variables in the EdGap data set. So let's check the minimum and maximum values in each column:

In [23]:
df.describe()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
count,7.986000e+03,7972.000000,7973.000000,7961.000000,7966.000000,7986.000000,7986.000000
mean,3.321869e+11,0.098730,0.568930,0.633440,52026.905222,20.181532,0.420651
std,1.323638e+11,0.058959,0.165704,0.196764,24228.057079,2.595201,0.239754
min,1.000016e+11,0.000000,0.091493,0.000000,3589.000000,-3.070818,-0.054545
25%,2.105340e+11,0.058655,0.450828,0.523810,36597.250000,18.600000,0.238501
50%,3.600085e+11,0.085649,0.554979,0.667594,46833.500000,20.400000,0.381570
75%,4.226678e+11,0.123376,0.676571,0.777135,61369.250000,21.910867,0.575447
max,5.605830e+11,0.590278,1.000000,1.000000,226181.000000,32.362637,0.998729


In [24]:
df.agg(['min', 'max']).round(2)

<ipython-input-24-39a2038683d4>:1: FutureWarning: ['year', 'state', 'zip_code', 'school_type', 'school_level'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.agg(['min', 'max']).round(2)


,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
min,100001600143,0.00,0.09,0.0,3589.0,-3.07,-0.05
max,560583000335,0.59,1.00,1.0,226181.0,32.36,1.00


In [25]:
df['state'].unique()

array(['DE', 'FL', 'GA', nan, 'IL', 'IN', 'KY', 'LA', 'MA', 'MI', 'MO',
       'NJ', 'NY', 'NC', 'OH', 'PA', 'TN', 'TX', 'WA', 'WI', 'WY'],
      dtype=object)

Now we will want to set out-of-range values to `NaN`.

In [26]:
df.loc[df['percent_lunch'] < 0, 'percent_lunch'] = np.nan

In [27]:
df.loc[df['average_act'] < 1, 'average_act'] = np.nan

Check the types of schools in the school information data set.

In [28]:
df['school_type'].value_counts()

Regular School                 7885
Alternative School               10
Special Education School          2
Career and Technical School       1
Name: school_type, dtype: int64

In [29]:
df['school_level'].value_counts()

High            7230
Other            631
Not reported      35
Elementary         2
Name: school_level, dtype: int64

We want to only keep the high schools.

In [30]:
df = df.loc[df['school_level'] == 'High']

## Identify missing values

Now we can start to identify the missing observations.

In [31]:
df.isna().sum()

id                    0
rate_unemployment    12
percent_college      11
percent_married      20
median_income        16
average_act           3
percent_lunch        20
year                  0
state                 0
zip_code              0
school_type           0
school_level          0
dtype: int64

What percentage of values of each variable are missing?

In [32]:
df.isna().mean().round(4)*100

id                   0.00
rate_unemployment    0.17
percent_college      0.15
percent_married      0.28
median_income        0.22
average_act          0.04
percent_lunch        0.28
year                 0.00
state                0.00
zip_code             0.00
school_type          0.00
school_level         0.00
dtype: float64

There are some schools that are missing all four of the socioeconomic variables. However, many of the schools are missing only a subset of the variables. If we drop rows that have `NaNs`, then we will negatively affect our analysis using the variables where data were present. So, we won't drop the rows in this data set that are missing the socioeconomic variables. 

We will, however, drop the rows where the average ACT score is missing.

We will impute the remaining missing values after splitting the data into training and testing sets for model evaluation.

In [33]:
df = df.loc[df['average_act'].isna() == False]

In [34]:
df.isna().sum()

id                    0
rate_unemployment    12
percent_college      11
percent_married      20
median_income        16
average_act           0
percent_lunch        20
year                  0
state                 0
zip_code              0
school_type           0
school_level          0
dtype: int64

## Train test split

#### Get input and output variables.

##### $\rightarrow$ Which variable is the output variable and which variables are input variables?

In [35]:
df.head(1)

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High


##### $\rightarrow$ We will define the matrix of predictor variables `X` to be all columns except `id` and `average_act` and define the output variable `y` to be `average_act`.

In [36]:
X = df[df.columns.difference(['id','average_act'])]
y = df['average_act']

In [37]:
X.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code
0,42820.0,0.445283,0.066901,0.346495,0.117962,High,Regular School,DE,2016-2017,19804
1,89320.0,0.662765,0.112412,0.767619,0.063984,High,Regular School,DE,2016-2017,19709
2,84140.0,0.701864,0.096816,0.713090,0.056460,High,Regular School,DE,2016-2017,19709
3,56500.0,0.692062,0.296960,0.641283,0.044739,High,Regular School,DE,2016-2017,19958
4,54015.0,0.640060,0.262641,0.834402,0.077014,High,Regular School,DE,2016-2017,19934


In [38]:
y.head()

0    20.433455
1    19.498168
2    19.554335
3    17.737485
4    18.245421
Name: average_act, dtype: float64

#### Train and test splits

We will use the test set approach to compare models predicting `average_act`. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/ML_dataset_training_validation_test_sets.png/800px-ML_dataset_training_validation_test_sets.png)

##### $\rightarrow$ Split the data into training and testing sets. Keep 20% of the data for the test set.

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state = 1)

In [40]:
print(X_train.shape, X_test.shape)

(5781, 10) (1446, 10)


##### $\rightarrow$ Check that the training and testing input variables have similar means and standard deviations.

In [41]:
X_train.agg(['mean','std']).round(2)

<ipython-input-41-288b8c217e1f>:1: FutureWarning: ['school_level', 'school_type', 'state', 'year', 'zip_code'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  X_train.agg(['mean','std']).round(2)


,median_income,percent_college,percent_lunch,percent_married,rate_unemployment
mean,53170.26,0.57,0.41,0.64,0.10
std,24827.94,0.17,0.23,0.19,0.06


In [42]:
X_test.agg(['mean','std']).round(2)

<ipython-input-42-247411c871de>:1: FutureWarning: ['school_level', 'school_type', 'state', 'year', 'zip_code'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  X_test.agg(['mean','std']).round(2)


,median_income,percent_college,percent_lunch,percent_married,rate_unemployment
mean,51179.23,0.57,0.42,0.63,0.10
std,22358.47,0.16,0.24,0.19,0.06


The distributions should be similar overall. We are looking at the mean and standard deviation for simplicity.

#### Imputation

We will use the iterative imputer in order to replace the NaN values.

In [43]:
imputer = IterativeImputer()

Now we fit the imputer using the training data only.

In [44]:
X_train.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code
3663,41793.0,0.602419,0.542056,0.574034,0.111111,High,Regular School,NJ,2016-2017,7306
1689,38173.0,0.469225,0.339655,0.711429,0.135246,High,Regular School,IN,2016-2017,47567
5852,39635.0,0.567361,0.270175,0.694514,0.083419,High,Regular School,PA,2016-2017,15853
3288,40978.0,0.467614,0.315556,0.766901,0.062531,High,Regular School,MO,2016-2017,64644
378,36875.0,0.604470,0.548410,0.803435,0.071429,High,Regular School,FL,2016-2017,34669


In [45]:
imputer.fit(X_train.loc[:,'median_income':'rate_unemployment'])

IterativeImputer()

Now we impute the missing values in the training data.

In [46]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5781 entries, 3663 to 5736
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   median_income      5766 non-null   float64
 1   percent_college    5770 non-null   float64
 2   percent_lunch      5764 non-null   float64
 3   percent_married    5763 non-null   float64
 4   rate_unemployment  5769 non-null   float64
 5   school_level       5781 non-null   object 
 6   school_type        5781 non-null   object 
 7   state              5781 non-null   object 
 8   year               5781 non-null   object 
 9   zip_code           5781 non-null   object 
dtypes: float64(5), object(5)
memory usage: 496.8+ KB


In [47]:
X_train.loc[:, 'median_income':'rate_unemployment'] = imputer.transform(X_train.loc[:,'median_income':'rate_unemployment'])

Check for missing values

In [48]:
X_train.isna().sum()

median_income        0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

Check for missing values in the test data

In [49]:
X_test.isna().sum()

median_income        1
percent_college      0
percent_lunch        3
percent_married      2
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

If there are missing values, use the imputer to replace the missing values.

In [50]:
X_test.loc[:,'median_income':'rate_unemployment'] = imputer.transform(X_test.loc[:,'median_income':'rate_unemployment'])

Check for missing values.

In [51]:
X_test.isna().sum()

median_income        0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

In [52]:
df_train = X_train.join(y_train)

In [53]:
df_train.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code,average_act
3663,41793.0,0.602419,0.542056,0.574034,0.111111,High,Regular School,NJ,2016-2017,7306,16.538462
1689,38173.0,0.469225,0.339655,0.711429,0.135246,High,Regular School,IN,2016-2017,47567,20.367521
5852,39635.0,0.567361,0.270175,0.694514,0.083419,High,Regular School,PA,2016-2017,15853,20.347985
3288,40978.0,0.467614,0.315556,0.766901,0.062531,High,Regular School,MO,2016-2017,64644,21.600000
378,36875.0,0.604470,0.548410,0.803435,0.071429,High,Regular School,FL,2016-2017,34669,21.056166


In [54]:
df_test = X_test.join(y_test)

In [55]:
df_test.head()

,median_income,percent_college,percent_lunch,percent_married,rate_unemployment,school_level,school_type,state,year,zip_code,average_act
2804,52833.0,0.564717,0.226481,0.823245,0.100518,High,Regular School,MI,2016-2017,49112,21.000000
4162,62411.0,0.537197,0.677895,0.313253,0.095582,High,Regular School,NY,2016-2017,11413,16.245421
5411,63938.0,0.781818,0.561431,0.523810,0.096433,High,Regular School,PA,2016-2017,15222,18.345543
4171,25625.0,0.361014,0.625239,0.317358,0.168471,High,Regular School,NY,2016-2017,11103,18.663004
1950,46350.0,0.602669,0.358377,0.641444,0.089737,High,Regular School,KY,2016-2017,42025,20.000000


In [62]:
df = df_test
df.isna().sum()

median_income        0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
average_act          0
dtype: int64

Now we can see that we have imputed and resolved the issue of there being missing values within the data frame. 

Now we have the desired information from the EdGap and School Information data frames in order for us to be able to start analyzing and determine whether socioeconomic factors play a role in school performance.

Now we can analyze and create new variables if necessary to help ease the analysis process to answer the bigger question of whether school performance is effected by socioeconomic factors.

## Export the clean .csv file

In [76]:
from google.colab import files

df.to_csv('clean_edgap_school_info.csv', encoding = 'utf-8-sig', index=False) 

files.download('clean_edgap_school_info.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>